In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
import time as time
from pymongo import MongoClient
from pymongo import CursorType

In [2]:
#   page의 주소가 1, 51, 101로 구분되어서 만들어주었다.

"""
def targeting(target):
    
    page_li = [] # 음원 page가 담길 list
    page_ = target // 50
    page_li.append(1)
    
    for i in range(1, page_):
        page_num = 1 + 50 * i
        page_li.append(page_num)
    
    return page_li
"""

    
def find_title(driver):
    title =  driver.find_elements_by_class_name('ellipsis.rank01')
    
    title2 = []

    for i in title:
        title2.append(i.text)
        
    return title2
    
    
def find_singer(driver):
    singer = driver.find_elements_by_class_name('ellipsis.rank02')

    singer2 = []

    for i in singer:
        singer2.append(i.text)
    
    return singer2
    
    
def next_page(driver, page):
    page_url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&steadyYn=Y#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=Y&po=pageObj&startIndex={}".format(page)
    driver.get(page_url)
    time.sleep(3)
      
    
def find_song_address(driver):
    
    song_address = driver.find_elements_by_class_name("input_check")
        
    song_address_li = []

    for i in song_address:
        song_address_li.append(i.get_attribute("value"))
        
    # 제대로 코드를 받아오는지 확인용 print
    print(song_address_li)
    time.sleep(3)
    
    return song_address_li


def lyric_crawl(driver, song_address_li):
    
    LYRIC = []
    
    for address in song_address_li[1:]:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + address)
            lyric = driver.find_element_by_class_name("lyric").text
            LYRIC.append(lyric.replace("\n", " ")) 
        except:
            LYRIC.append("가사없음")
     
    time.sleep(5)
    return LYRIC



def genre(driver, song_address_li):
     
    GENRE = []
        
    for address in song_address_li[1:]:
        
        driver.get("https://www.melon.com/song/detail.htm?songId=" + address)
        genre = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text
        GENRE.append(genre)
        time.sleep(5)
        
    return GENRE 
        

In [3]:
def insert_item_one(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_one(dates).inserted_id
    return result

def insert_item_many(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_many(dates).inserted_ids
    return result

def find_item_one(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find_one(condition, {"_id": False})
    return result

def find_item(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find(condition, {"_id":False}, no_cursor_timeout=True, cursor_type=CursorType.EXHAUST)
    return result

In [4]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
options = webdriver.ChromeOptions()
        
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# options.add_argument('headless')

driver = webdriver.Chrome('chromedriver.exe',options=options)
driver.get('https://member.melon.com/muid/web/login/login_informM.htm')

username = 'kgs8291'
password = 'wodud3558!'

element_id = driver.find_element_by_name("id")
element_id.send_keys(username)
element_password = driver.find_element_by_name("pwd")
element_password.send_keys(password)


driver.find_element_by_class_name("btn_login03").click()

time.sleep(3)

# 크롤링 시작할 곳
driver.get('https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&steadyYn=Y')

In [5]:
host = "localhost"
port = "27017"
mongo = MongoClient(host, int(port))

lyric_li = []
singer_li = []
title_li = []
genre_li = []

num = 1

# page_li_index = targeting(50)
try:
    for _ in range(14):
        next_page( driver= driver, page=num) # 페이지 이동
        title_li.append(find_title(driver= driver)) # 노래 제목
        singer_li.append(find_singer(driver= driver)) # 가수
        song_address_list = find_song_address(driver = driver) # 노래 주소 반환
        genre_li.append(genre(driver= driver, song_address_li= song_address_list)) # 장르 반환
        lyric_li.append(lyric_crawl(driver= driver, song_address_li= song_address_list)) # 노래 가사 반환
    
        num += 50
        
except Exception as ex:
    print('에러가 발생 했습니다', ex)
    pass

    
# 몽고 디비에 입력
for i in range(len(title_li)):
    for j in range(len(title_li[0])):
        insert_item_one(mongo, {"genre": "{}".format(genre_li[i][j]), "title": "{}".format(title_li[i][j]), "singer" : "{}".format(singer_li[i][j]), "lyric" : "{}".format(lyric_li[i][j])},"music","dance")
        
        

['on', '7891119', '5791706', '7896508', '3053259', '3051244', '5748406', '5748499', '3625504', '5743804', '1325050', '1951556', '7847861', '3795586', '7956303', '7956304', '2212365', '3715676', '4562881', '4756304', '4764928', '62462', '3837683', '5629148', '2093360', '5721440', '1951557', '1756975', '4268435', '81013', '5820370', '3929469', '91310', '71581', '4193337', '5826461', '7935396', '2558738', '5820368', '5644620', '608151', '1897011', '71606', '5660045', '503339', '1951550', '2930968', '4711007', '10434', '2371269', '3488859']
['on', '5546489', '3521672', '4163116', '5767428', '622664', '1928828', '5473905', '70994', '5781104', '7896507', '38547', '3457172', '3509750', '2258761', '3715672', '5674473', '2229259', '1322134', '1140592', '4171462', '2218821', '1724693', '1874397', '4840373', '191093', '5644636', '4454382', '4400561', '5472103', '4806688', '4633162', '3468048', '3403761', '83721', '38426', '1903050', '2558739', '1927248', '5658708', '3734735', '5456521', '1929329'

IndexError: list index out of range